(!) **Note**: We want to compare polarised flux to total intensity flux

    i.e. Fobject,pol/Fstar,pol and Fobject,ti/Fobject,ti
    
Below, we can see that flux of the star in ADU/s is higher in polar than in total intensity when considering one channel for the total intensity. I think it's ok because in polar we always consider both left+right detectors, contrary to total intensity with only one channel.

In [2]:
from import_functions_generic import * 
from import_functions_plotting import * 
#from image_manipulation import * 

In [3]:
L = time.localtime()
date = "{}-{}-{}".format(L[0],L[1],L[2],L[3],L[4],L[5])

saving_dir = 'figs/fig_v{}/'.format(date)
os.makedirs(saving_dir,exist_ok=True)

# 1. From (forward-)modelling results 


**Forward modelling results:**

Inputs: 
- IRDAP image (ADU) and associated PSF normalized to 1
- PCA image (ADU) and associated PSF normalized to 1

So in principle I would only need to correct the fitted scaling values by DIT and TRANSMISSION.

Note: the PSF is normalized in the FM code as follows: sum over all pixels with a PSF of shape (25,25) in polar (value = 723584) and (25,25) in total intensity (value = 9028321), so consistent!

DO THAT TOMORROW's MORNING


By modelling the disk we want to minimize the residuals expressed as follows

$$    \text{residuals} \;=\; \frac{\text{reduced_obs_image} \times \text{mask} -\text{synthetic_model_disk}*\text{PSF}}{\text{noise}}\,. $$

with

- the sum of the PSF values are normalized to 1

- the mask is a binary mask with 0 or 1 values

- '$*$' stands for convolution

The model of the disk is computed with the $\texttt{VIP}$ function  $\texttt{ScatteredLightDisk(...)}$ and then multiply by a $scaling$ parameter that is fitted with the $\texttt{AMOEBA}$ exploration of the parameter space, i.e.

$$\text{synthetic_model_disk} = \texttt{ScatteredLightDisk(...)} \times scaling$$

**Notes**
as the PSF is normalized to 1, the total flux of the disk or the convolved disk is the same.



# Polarimetry

**Goal:** From Qphi image, we have values in ADU. We want to convert this ADU value in a flux value.

Correction from time exposure and transmission is simply

$$\text{IM (ADU)} \; \rightarrow\;  \frac{1}{\text{DIT}} \times \frac{1}{\text{TRANSMISSION}}  \times \text{IM (ADU)} $$  

If the image QPHI is corrected from transmission and time exposure, the flux measured in ADU value can be converted in Jy as follows


$$\text{QPHI (ADU)} \; \rightarrow\; \text{QPHI (Jy)}\; =\; \frac{\text{QPHI (ADU)}}{\text{STAR (ADU)}}  \times \text{STAR (Jy)} $$ 

If the image QPHI is *not yet* corrected from transmission and time exposure, the flux measured in ADU value can be converted in Jy as follows

$$\text{QPHI (ADU)} \; \rightarrow\; \text{QPHI (Jy)}\; =\; \frac{\text{QPHI (ADU)}}{\text{STAR (ADU)}} \times \frac{\text{DIT}_\star}{\text{DIT}_\text{QPHI}} \times \frac{\text{TRANSMISSION}_\star}{\text{TRANSMISSION}_\text{QPHI}}  \times \text{STAR (Jy)} $$ 



In [29]:
# Corresponding function

def ADU2FLUX(flux_adu_obj, flux_adu_star, flux_ref_star, dit_obj, dit_star, transmi_obj, transmi_star, platescale):
    x = flux_adu_star / (dit_star * transmi_star) 
    #print('Star flux = %.3e ADU/s' % x )
    return flux_adu_obj/ flux_adu_star * dit_star/dit_obj * transmi_star/transmi_obj * flux_ref_star 


def ADU2FRACTION(flux_adu_obj, flux_adu_star, flux_ref_star, dit_obj, dit_star, transmi_obj, transmi_star, platescale):
    x = flux_adu_star / (dit_star * transmi_star) 
    #print('Star flux = %.3e ADU/s' % x )
    return flux_adu_obj/ flux_adu_star * dit_star/dit_obj * transmi_star/transmi_obj

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [30]:
## Params: obs
DIT_STAR, DIT_OBJECT = 0.837464, 64 # s taken from the header of the images
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.12591487665785145, 1 # corresponds to ND1.0 and OPEN; taken from the header of the images
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975 # Jy from Simbad in the filter Johnson H (!!)

#### Results from fitting only polarimetry obs

In [32]:
## Params: flux
scaling = 0 # fitted parameter; not used
flux_adu_obj = 17669 # total flux of the disk after multiplying by the scaling parameter; this corresponds the disk_convolved image
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('\nThe flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

fraction_polar_ref_indpt = flux_obj_final_unit

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 17669.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk is 276.08 ADU/s

The flux fraction Fd/Fstar is 4.02e-05.
The flux of the disk is 3.92e-05 Jy.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, 1 channel

In [9]:
## Params: flux
scaling = 1514654 # fitted parameter; not used
flux_adu_obj = 17656 #18794 # total flux of the disk after multiplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('\nThe flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

fraction_polar_ref_1TI_5modes_1channel = flux_obj_final_unit 

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 17656.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 275.88 ADU/s

The flux fraction Fd/Fstar is 4.02e-05.
The flux of the disk is 3.92e-05 Jy.


##### for the total intensity-part: used 5 modes, both channels

In [10]:
## Params: flux
scaling = 1593331 # fitted parameter; not used
flux_adu_obj = 21806 #18838 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.24e+05 ADU.
The flux of the disk is 21806.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk is 340.72 ADU/s


In [11]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)
print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

fraction_polar_ref_1TI_5modes_2channel = flux_obj_final_unit 

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 4.97e-05.
The flux of the disk is 4.84e-05 Jy.


##### for the total intensity-part: used 10 modes, 1 channel

In [12]:
## Params: flux
scaling = 1226197 # fitted parameter; not used
flux_adu_obj = 19046 #18453 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 19046.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 297.59 ADU/s


In [13]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

fraction_polar_ref_1TI_10modes_1channel = flux_obj_final_unit 

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 4.34e-05.
The flux of the disk is 4.23e-05 Jy.


##### for the total intensity-part: used 10 modes, both channels

In [14]:
## Params: flux
scaling = 1421607 # fitted parameter; not used
flux_adu_obj = 17752 #18749 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 17752.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 277.38 ADU/s


In [16]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

fraction_polar_ref_1TI_10modes_2channel = flux_obj_final_unit 

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 4.04e-05.
The flux of the disk is 3.94e-05 Jy.


# Total intensity

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We take the mean from before and after the observation and consider either only the left or right cubes, or the sum of them, depending if we work on one or the channels of the SCIENCE cube.

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

**Warning note :** specal convert outputs are already corrected from the DIT and the neutral density !

In [17]:
# Params
DIT_STAR, DIT_OBJECT = 8, 32 # s
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.014581900, 1 # ND2.0 and OPEN are used. Corresponding value for ND2.0
# taken from the ESO website https://www.eso.org/sci/facilities/paranal/instruments/sphere/inst/filters.html
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975  # Jy from Simbad in the filter Johnson H


# Params
DIT_STAR, DIT_OBJECT = 1, 1 # s
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 1, 1 # ND2.0 and OPEN are used. Corresponding value for ND2.0
# taken from the ESO website https://www.eso.org/sci/facilities/paranal/instruments/sphere/inst/filters.html
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975  # Jy from Simbad in the filter Johnson H

#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, 1 channel

In [18]:
## Params: flux
scaling = 35124 # fitted parameter; not used
flux_adu_obj = 4995 #4924 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 9028321/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))


(results *not* corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU.
The flux of the disk is 4995.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 4.51e+06 ADU/s.
The flux of the disk is 4995.00 ADU/s


In [20]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref_1TI_5modes_1channel/flux_obj_final_unit))

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 1.11e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.036.
The flux of the disk is 1.08e-03 Jy.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, both channels

In [21]:
## Params: flux
scaling = 73490 # fitted parameter; not used
flux_adu_obj = 18454 #10645 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 18083288/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU.
The flux of the disk is 18454.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 9.04e+06 ADU/s.
The flux of the disk is 18454.00 ADU/s


In [22]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref_1TI_5modes_2channel/flux_obj_final_unit))

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 2.04e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.024.
The flux of the disk is 1.99e-03 Jy.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 10 modes, 1 channel

In [23]:
## Params: flux
scaling = 26412 # fitted parameter; not used
flux_adu_obj = 3563 #3545 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 9028321/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU.
The flux of the disk is 3563.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU/s.
The flux of the disk is 3563.00 ADU/s


In [24]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref_1TI_10modes_1channel/flux_obj_final_unit))

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 7.89e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.055.
The flux of the disk is 7.70e-04 Jy.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 10 modes, both channels

In [25]:
## Params: flux
scaling = 56923 # fitted parameter; not used
flux_adu_obj = 7925 #7638 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 18083288/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))
 
print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU.
The flux of the disk is 7925.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 9.04e+06 ADU/s.
The flux of the disk is 7925.00 ADU/s


In [26]:
flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref_1TI_10modes_2channel/flux_obj_final_unit))

flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))

The flux fraction Fd/Fstar is 8.76e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.046.
The flux of the disk is 8.55e-04 Jy.


### Results from independent FM 

##### for the total intensity-part: used 5 modes, 1 channel

In [33]:
fraction_polar_ref = fraction_polar_ref_indpt # independent fit

In [8]:
#DIT_OBJECT_S = [32, 32, 32]
#DIT_STAR_S   = [8, 4.001856, 4.001856]

#TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [8285, 1465, 1715]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421] #(25,25) #[8245767/2, 1647963/2, 844699] (15,15)
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [9]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))
    
    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))


    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))
    


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.51e+06 ADU/s.
The flux of the disk is 8285.00 ADU/s
The flux fraction Fd/Fstar is 1.84e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.022.
The flux of the disk is 1.79e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.84e+05 ADU/s.
The flux of the disk is 1465.00 ADU/s
The flux fraction Fd/Fstar is 1.66e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.024.
The flux of the disk is 1.62e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.25e+05 ADU/s.
The flux of the disk is 1715.00 ADU/s
The flux fraction Fd/Fstar is 1.85e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.022.
The flux of the disk is 1.81e-03 Jy.


##### for the total intensity-part: used 5 modes, both channels

In [10]:
DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [3798, 2158, 1898]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827] #[16512481/2, 3300536/2, ]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [11]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))
    
    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
     
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))


    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.04e+06 ADU/s.
The flux of the disk is 3798.00 ADU/s
The flux fraction Fd/Fstar is 4.20e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.096.
The flux of the disk is 4.10e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.78e+06 ADU/s.
The flux of the disk is 2158.00 ADU/s
The flux fraction Fd/Fstar is 1.21e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.033.
The flux of the disk is 1.18e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.84e+06 ADU/s.
The flux of the disk is 1898.00 ADU/s
The flux fraction Fd/Fstar is 1.03e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.039.
The flux of the disk is 1.01e-03 Jy.


##### for the total intensity-part: used 10 modes, 1 channel

In [34]:
DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]


DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

Channel = 2

if Channel == 1 :
    FLUX_ADU_OBJ  = [3084, 1494, 1603]
    FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421] # [8245767/2, 1647963/2, 844699]
    
else:
    FLUX_ADU_OBJ  = [8285, 1560, 1340]
    FLUX_ADU_STAR = [9054966/2, 1788593/2, 911405] #(25,25) #[8245767/2, 1647963/2, 844699] (15,15)
    

INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [35]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
      
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.53e+06 ADU/s.
The flux of the disk is 8285.00 ADU/s
The flux fraction Fd/Fstar is 1.83e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.022.
The flux of the disk is 1.78e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.94e+05 ADU/s.
The flux of the disk is 1560.00 ADU/s
The flux fraction Fd/Fstar is 1.74e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.023.
The flux of the disk is 1.70e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.11e+05 ADU/s.
The flux of the disk is 1340.00 ADU/s
The flux fraction Fd/Fstar is 1.47e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.027.
The flux of the disk is 1.43e-03 Jy.


##### for the total intensity-part: used 10 modes, both channels

In [14]:
DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]



DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [9108, 2262, 1339]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827] #[16512481/2, 3300536/2, ]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [15]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.4e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
      
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.4e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.0416e+06 ADU/s.
The flux of the disk is 9108.00 ADU/s
The flux fraction Fd/Fstar is 1.01e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.040.
The flux of the disk is 9.8216e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.7785e+06 ADU/s.
The flux of the disk is 2262.00 ADU/s
The flux fraction Fd/Fstar is 1.27e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.032.
The flux of the disk is 1.2400e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8368e+06 ADU/s.
The flux of the disk is 1339.00 ADU/s
The flux fraction Fd/Fstar is 7.29e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.055.
The flux of the disk is 7.1075e-04 Jy.


# Polarimetry & Total intensity -- concatenated results

### Results from joint-fitting polarimetry+total intensity in both BBH and H23 bands 

##### for the total intensity-part: used 5 modes, 1 channel

In [290]:
DIT_OBJECT_S = [32, 32, 32, 64]
DIT_STAR_S   = [8, 4.001856, 4.001856, 0.837464]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]


DIT_OBJECT_S = [1, 1, 1, 64]
DIT_STAR_S   = [1, 1, 1, 0.837464]

TRANSMISSION_STAR_S = [1, 1, 1, 0.12591487665785145]

FLUX_ADU_OBJ  = [5043, 907.7, 644.4, 18831]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [292]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))
    
    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
    # should be run twice due to the loop order
    if i ==3: fraction_polar_ref = flux_obj_final_unit
    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.51e+06 ADU/s.
The flux of the disk is 5043.00 ADU/s
The flux fraction Fd/Fstar is 1.12e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.038.
The flux of the disk is 1.09e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.84e+05 ADU/s.
The flux of the disk is 907.70 ADU/s
The flux fraction Fd/Fstar is 1.03e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.042.
The flux of the disk is 1.00e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.25e+05 ADU/s.
The flux of the disk is 644.40 ADU/s
The flux fraction Fd/Fstar is 6.96e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.062.
The flux of the disk is 6.79e-04 Jy.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk i

##### for the total intensity-part: used 5 modes, both channels

In [293]:
DIT_OBJECT_S = [32, 32, 32, 64]
DIT_STAR_S   = [8, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

DIT_OBJECT_S = [1, 1, 1, 64]
DIT_STAR_S   = [1, 1, 1, 0.837464]

TRANSMISSION_STAR_S = [1, 1, 1, 0.12591487665785145]

FLUX_ADU_OBJ  = [8806, 1442, 1533, 18073]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [295]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    
    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

    # should be run twice due to the loop order
    if i ==3: fraction_polar_ref = flux_obj_final_unit
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU/s.
The flux of the disk is 8806.00 ADU/s
The flux fraction Fd/Fstar is 9.74e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.042.
The flux of the disk is 9.50e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8e+06 ADU/s.
The flux of the disk is 1442.00 ADU/s
The flux fraction Fd/Fstar is 8.11e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.051.
The flux of the disk is 7.91e-04 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8e+06 ADU/s.
The flux of the disk is 1533.00 ADU/s
The flux fraction Fd/Fstar is 8.35e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.049.
The flux of the disk is 8.14e-04 Jy.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 

##### for the total intensity-part: used 10 modes, 1 channel

In [322]:
DIT_OBJECT_S = [32, 32, 32, 64]
DIT_STAR_S   = [8, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

DIT_OBJECT_S = [1, 1, 1, 64]
DIT_STAR_S   = [1, 1, 1, 0.837464]

TRANSMISSION_STAR_S = [1, 1, 1, 0.12591487665785145]

FLUX_ADU_OBJ  = [3290, 709.3, 711.1, 18162]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [323]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)
    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

    # should be run twice due to the loop order
    if i ==3: fraction_polar_ref = flux_obj_final_unit    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.51e+06 ADU/s.
The flux of the disk is 3290.00 ADU/s
The flux fraction Fd/Fstar is 7.29e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.057.
The flux of the disk is 7.11e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.84e+05 ADU/s.
The flux of the disk is 709.30 ADU/s
The flux fraction Fd/Fstar is 8.02e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.052.
The flux of the disk is 7.82e-04 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.25e+05 ADU/s.
The flux of the disk is 711.10 ADU/s
The flux fraction Fd/Fstar is 7.68e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.054.
The flux of the disk is 7.49e-04 Jy.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk i

##### for the total intensity-part: used 10 modes, both channels

In [327]:
DIT_OBJECT_S = [32, 4.001856, 4.001856, 64]
DIT_STAR_S   = [32, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

DIT_OBJECT_S = [1, 1, 1, 64]
DIT_STAR_S   = [1, 1, 1, 0.837464]

TRANSMISSION_STAR_S = [1, 1, 1, 0.12591487665785145]

FLUX_ADU_OBJ  = [6709, 1365, 1113, 18173]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
#FLUX_ADU_STAR = [9041644.0, 1778532.9, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [328]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.4e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)
    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

    # should be run twice due to the loop order
    if i ==3: fraction_polar_ref = flux_obj_final_unit    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

          
    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.0416e+06 ADU/s.
The flux of the disk is 6709.00 ADU/s
The flux fraction Fd/Fstar is 7.42e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.056.
The flux of the disk is 7.23e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.7785e+06 ADU/s.
The flux of the disk is 1365.00 ADU/s
The flux fraction Fd/Fstar is 7.67e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.054.
The flux of the disk is 7.48e-04 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8368e+06 ADU/s.
The flux of the disk is 1113.00 ADU/s
The flux fraction Fd/Fstar is 6.06e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.068.
The flux of the disk is 5.91e-04 Jy.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.8619e+06 ADU/s.
The flux of 

### Results from joint-fitting total intensity in both BBH and H23 bands 

##### for the total intensity-part: used 5 modes, 1 channel

In [329]:
fraction_polar_ref = fraction_polar_ref_indpt

DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [8819, 1231, 1691]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [331]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.51e+06 ADU/s.
The flux of the disk is 8819.00 ADU/s
The flux fraction Fd/Fstar is 1.95e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.022.
The flux of the disk is 1.90e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.84e+05 ADU/s.
The flux of the disk is 1231.00 ADU/s
The flux fraction Fd/Fstar is 1.39e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.030.
The flux of the disk is 1.36e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.25e+05 ADU/s.
The flux of the disk is 1691.00 ADU/s
The flux fraction Fd/Fstar is 1.83e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.023.
The flux of the disk is 1.78e-03 Jy.


##### for the total intensity-part: used 5 modes, both channels

In [335]:
DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [18956, 3294, 3043]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [336]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))
    
    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.04e+06 ADU/s.
The flux of the disk is 18956.00 ADU/s
The flux fraction Fd/Fstar is 2.10e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.020.
The flux of the disk is 2.04e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.78e+06 ADU/s.
The flux of the disk is 3294.00 ADU/s
The flux fraction Fd/Fstar is 1.85e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.023.
The flux of the disk is 1.81e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.84e+06 ADU/s.
The flux of the disk is 3043.00 ADU/s
The flux fraction Fd/Fstar is 1.66e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.026.
The flux of the disk is 1.62e-03 Jy.


##### for the total intensity-part: used 10 modes, 1 channel

In [332]:
DIT_OBJECT_S = [32, 32, 32]
DIT_STAR_S   = [8, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]


DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [5000, 1176, 1071]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [333]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))

    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU/s.
The flux of the disk is 5000.00 ADU/s
The flux fraction Fd/Fstar is 1.11e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.038.
The flux of the disk is 1.08e-03 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 8.8e+05 ADU/s.
The flux of the disk is 1176.00 ADU/s
The flux fraction Fd/Fstar is 1.33e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.032.
The flux of the disk is 1.30e-03 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 9.3e+05 ADU/s.
The flux of the disk is 1071.00 ADU/s
The flux fraction Fd/Fstar is 1.16e-03.
The polarised fraction of the disk Fd_polar/Fd is 0.037.
The flux of the disk is 1.13e-03 Jy.


##### for the total intensity-part: used 10 modes, both channels

In [310]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]


DIT_OBJECT_S = [1, 1, 1]
DIT_STAR_S   = [1, 1, 1]

TRANSMISSION_STAR_S = [1, 1, 1]

FLUX_ADU_OBJ  = [8510, 1728, 1201]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [311]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FRACTION(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux fraction Fd/Fstar is {:.2e}.'.format(flux_obj_final_unit))
    
    print('The polarised fraction of the disk Fd_polar/Fd is {:.3f}.'.format(fraction_polar_ref/flux_obj_final_unit))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU/s.
The flux of the disk is 8510.00 ADU/s
The flux fraction Fd/Fstar is 9.41e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.045.
The flux of the disk is 9.18e-04 Jy.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8e+06 ADU/s.
The flux of the disk is 1728.00 ADU/s
The flux fraction Fd/Fstar is 9.72e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.044.
The flux of the disk is 9.47e-04 Jy.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.8e+06 ADU/s.
The flux of the disk is 1201.00 ADU/s
The flux fraction Fd/Fstar is 6.54e-04.
The polarised fraction of the disk Fd_polar/Fd is 0.065.
The flux of the disk is 6.37e-04 Jy.


# 2. From processed data

From polarimetry maybe relevant but be careful for total imaging: self-subtraction effect !! so underestimation of the flux on the processed image.


# Polarimetry

**Goal:** From Qphi image, we have values in ADU We want to convert this ADU value in a flux value.


$$\text{QPHI (ADU)} \; \rightarrow\; \text{QPHI (Jy)}\; =\; \frac{\text{QPHI (ADU)}}{\text{STAR (ADU)}} \times \frac{\text{DIT}_\star}{\text{DIT}_\text{QPHI}} \times \frac{\text{TRANSMISSION}_\star}{\text{TRANSMISSION}_\text{QPHI}}  \times \frac{1}{\text{PLATESCALE (arcsec/pixel)}^2} \times \text{STAR (Jy)} $$  

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [48]:
## Params: obs
DIT_STAR, DIT_OBJECT = 0.837464, 64 # s taken from the header of the images
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.12591487665785145, 1 # corresponds to ND1.0 and OPEN; taken from the header of the images
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975 # Jy from Simbad in the filter Johnson H

In [49]:
#DIT_STAR/DIT_OBJECT

In [50]:
#TRANSMISSION_STAR/TRANSMISSION_OBJECT

Note: for the polarimetric image, IRDAP considers both left and right cubes. Thus, when we consider the stellar flux, we sum the values of the left and right images within a circular of a given radius.

By default, a radius of 120 pixels is used in the IRDAP processing. However, as we would like to compare polarised fraction to total intensity fraction, and that in total intensity the PSF file ("median_unsat.fits") is smaller (64x64 pixels^2), we use a radius of 30 pixels to derive the stellar flux in polarimetry and in total intensity. 



In [31]:
## Params: flux
flux_adu_obj = 25 # value of a pixel in a bright location of the disk in the QPHI IRDAP-processed image.
flux_adu_star = (194015+195473) # within circular region of r = 30 pixels, left+right images
#flux_adu_star = 216706 # within circular region of r = 120 pixels. Default is r = 120 pixels.


print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s(!)'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 3.9e+05 ADU.
The flux of the disk is 25.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 3.7e+06 ADU/s.
The flux of the disk is 0.39 ADU/s(!)


In [32]:
flux_obj_final_unit =  ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 3.694e+06 ADU/s
The flux of the disk is 6.9e-04 Jy/arcsec^2.


# Total intensity

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We sum the flux of the left and rights images, and take the mean from before and after the observation.

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [33]:
# Params
DIT_STAR, DIT_OBJECT = 8, 32 # s
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.014581900, 1 # ND2.0 and OPEN are used. Corresponding value for ND2.0
# taken from the ESO website https://www.eso.org/sci/facilities/paranal/instruments/sphere/inst/filters.html
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975  # Jy from Simbad in the filter Johnson H

#### Consider only one side of the detector (indexed as 0; first channel)

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We consider the psf only on one part of the cube (indexed as 0; first channel), and take the mean from before and after the observation.

In [37]:
# STAR
# within circular region of r = 30 pixels centered in (33,33)
flux_adu_star = np.array( [[5107827, 5137693], [4738552, 4768522]]) # within box region of size = 30 pixels centered in (33,33)
flux_adu_star = np.nanmean( flux_adu_star[:,0] )


# The science reduced image is given in contrast; multiply by a normalization factor to have it back in ADU
normalization_factor = np.array([7774262.5, 7792016]) 
normalization_factor = np.mean(normalization_factor)

flux_adu_obj = 7.62037e-08 * normalization_factor

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU(!)'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s(!)'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 4.9e+06 ADU.
The flux of the disk is 0.59 ADU(!)

(results corrected from DITs and transmission)
The flux of the star is 4.2e+07 ADU/s.
The flux of the disk is 0.02 ADU/s(!)


Weird that the flux value of the disk in total intensity is lower than the flux value derived from polarimetry. From a factor 20 (!)  
Célia's guess: The flux value derived for the disk suffers from self-subtraction.

In [23]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 4.220e+07 ADU/s
The flux of the disk is 2.9e-06 Jy/arcsec^2.


#### Sum over the two wavelengths

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We sum the flux of the left and rights images, and take the mean from before and after the observation.

In [152]:
# STAR
# within circular region of r = 30 pixels centered in (33,33)
flux_adu_star = np.array( [[5107827, 5137693], [4738552, 4768522]]) # within box region of size = 30 pixels centered in (33,33)
flux_adu_star = np.nanmean( np.nansum(flux_adu_star, axis=1) )

print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))

# The science reduced image is given in contrast; multiply by a normalization factor to have it back in ADU
normalization_factor = np.array([7774262.5, 7792016]) 
normalization_factor = np.mean(normalization_factor)

flux_adu_obj = 7.62037e-08 * normalization_factor * 2 # multiply by 2 because consider two bands so twice photons more
print('The flux of the disk is {:.2f} ADU (!)'.format(flux_adu_obj))

The flux of the star is 9.9e+06 ADU.
The flux of the disk is 1.19 ADU (!)


Weird that the value is so low compared to polarimetry?!

In [153]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 8.466e+07 ADU/s
The flux of the disk is 2.8e-06 Jy/arcsec^2.


In [154]:
flux_adu_star

9876297.0

# Test

Comparison PCA processed images, 10 modes, pixel (296,308) ds9 convention with normalization (by the PSF, contrast values) or without normalization (still ADU)

In [42]:
flux_1 = 1.1959697
flux_norm = 1.1825136e-7 
flux_1/flux_norm

10113792.348772988

In [46]:
flux_1/normalization_factor * 1/flux_norm

1.2994489786075698

In [ ]:
flux_adu_star/(flux_1/flux_norm)

In [45]:
0.092981704/2.3942583e-8

3883528.5232173987

Unit("W / m2")